<a href="https://colab.research.google.com/github/OneFineStarstuff/Pinn/blob/main/SecureAGISystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import json
import yaml
import hashlib
import logging
from datetime import datetime
from typing import Optional, Tuple, Dict, Any
from cryptography.fernet import Fernet

import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.distribute import MultiWorkerMirroredStrategy

import matplotlib.pyplot as plt
import boto3

# --- Custom Exceptions ---
class ModelInitializationError(Exception): pass
class ModelNotFoundError(Exception): pass

class SecureAGISystem:
    def __init__(self, config_path: str, model_loader, distributed_training: bool = False, encryption_key: Optional[bytes] = None):
        self.config = self._load_config(config_path)
        self.models: Dict[str, Model] = {}
        self.model_loader = model_loader
        self.distributed_training = distributed_training
        self.strategy = self._initialize_strategy()
        self.encryption_key = encryption_key
        self.fernet = Fernet(encryption_key) if encryption_key else None

        os.makedirs("models", exist_ok=True)
        os.makedirs("logs", exist_ok=True)
        os.makedirs("reports", exist_ok=True)
        os.makedirs("checkpoints", exist_ok=True)

        logging.basicConfig(filename="training_audit.log", level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        logging.info("SecureAGISystem initialized.")

    def _load_config(self, path: str) -> Dict[str, Any]:
        with open(path, 'r') as f:
            return yaml.safe_load(f) if path.endswith(".yaml") else json.load(f)

    def _initialize_strategy(self):
        if self.distributed_training:
            return MultiWorkerMirroredStrategy()
        return None

    def _compute_dataset_hash(self, data: np.ndarray) -> str:
        return hashlib.sha256(data.tobytes()).hexdigest()

    def validate_dataset(self, x_train, y_train, expected_hash: Optional[str] = None):
        current_hash = self._compute_dataset_hash(x_train) + self._compute_dataset_hash(y_train)
        if expected_hash and current_hash != expected_hash:
            raise ValueError("Dataset integrity check failed!")
        return current_hash

    def get_model(self, model_name: str) -> Model:
        if model_name in self.models:
            return self.models[model_name]
        model = self.model_loader(model_name)
        if not isinstance(model, Model):
            raise ModelNotFoundError(f"No valid model for '{model_name}'")
        self.models[model_name] = model
        return model

    def save_model(self, model: Model, model_name: str, metrics: Dict[str, Any]):
        version = datetime.now().strftime("%Y%m%d-%H%M%S")
        save_path = os.path.join("models", model_name, version)
        os.makedirs(save_path, exist_ok=True)
        model.save(save_path)

        if self.fernet:
            for file in os.listdir(save_path):
                file_path = os.path.join(save_path, file)
                with open(file_path, 'rb') as f:
                    encrypted = self.fernet.encrypt(f.read())
                with open(file_path, 'wb') as f:
                    f.write(encrypted)

        metadata = {"model_name": model_name, "version": version, "metrics": metrics}
        with open(os.path.join(save_path, "metadata.json"), "w") as f:
            json.dump(metadata, f)

        logging.info(f"Model saved: {metadata}")
        return save_path, version

    def generate_training_report(self, history, metadata, save_path):
        plt.figure(figsize=(10,5))
        for key in history.history:
            plt.plot(history.history[key], label=key)
        plt.legend()
        plt.title("Training Metrics")
        plt.xlabel("Epoch")
        plt.ylabel("Value")
        plot_path = save_path.replace(".html", "_plot.png")
        plt.savefig(plot_path)
        plt.close()

        html_content = f"""
        <html>
        <head><title>Training Report</title></head>
        <body>
            <h1>Model Training Report</h1>
            <h2>Metadata</h2>
            <pre>{json.dumps(metadata, indent=4)}</pre>
            <h2>Training Curves</h2>
            <img src="{os.path.basename(plot_path)}" alt="Training Plot">
        </body>
        </html>
        """
        with open(save_path, "w") as f:
            f.write(html_content)

    def upload_to_s3(self, local_path, bucket_name, s3_path):
        s3 = boto3.client("s3")
        for root, dirs, files in os.walk(local_path):
            for file in files:
                full_path = os.path.join(root, file)
                rel_path = os.path.relpath(full_path, local_path)
                s3.upload_file(full_path, bucket_name, os.path.join(s3_path, rel_path))

    def train(self, model_name: str, data: Tuple[np.ndarray, np.ndarray], val_data: Optional[Tuple[np.ndarray, np.ndarray]] = None):
        cfg = self.config["training"]

        dataset_hash = self.validate_dataset(data[0], data[1], self.config.get("expected_dataset_hash"))

        log_dir = os.path.join("logs", "fit", f"{model_name}_{datetime.now().strftime('%Y%m%d-%H%M%S')}")
        callbacks = [
            TensorBoard(log_dir=log_dir, histogram_freq=1),
            ModelCheckpoint(filepath=f"checkpoints/{model_name}.h5", save_best_only=True)
        ]

        if self.strategy:
            with self.strategy.scope():
                model = self.get_model(model_name)
                model.compile(optimizer=cfg["optimizer"], loss=cfg["loss"], metrics=cfg["metrics"])
        else:
            model = self.get_model(model_name)
            model.compile(optimizer=cfg["optimizer"], loss=cfg["loss"], metrics=cfg["metrics"])

        history = model.fit(
            data[0], data[1],
            epochs=cfg["epochs"],
            batch_size=cfg["batch_size"],
            validation_data=val_data,
            callbacks=callbacks
        )

        final_metrics = {m: history.history[m][-1] for m in history.history}
        save_path, version = self.save_model(model, model_name, final_metrics)

        metadata = {"model_name": model_name, "dataset_hash": dataset_hash, "final_metrics": final_metrics, "version": version}
        report_path = f"reports/{model_name}_{version}.html"
        self.generate_training_report(history, metadata, report_path)

        if self.config.get("cloud_upload"):
            self.upload_to_s3(save_path, self.config["s3_bucket"], f"{model_name}/{version}")